In [102]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import StandardScaler

In [253]:
data = pd.read_csv("../data/combined.csv") 
data_values = data[["energia","proteina","grasa","carbohidratos"]]
nutritional_values = pd.read_csv("../data/nutritional_values.csv")

In [254]:
scaler = StandardScaler()
data[target_columns] = scaler.fit_transform(data[target_columns])
data

,Recipe1,Recipe2,energia,grasa,proteina,carbohidratos
0,51,53,-1.440671,-0.104548,-1.335397,-1.261165
1,51,61,-1.128588,-0.325347,-0.507423,-1.168580
2,51,68,1.209545,1.890373,-0.480118,-0.075447
3,51,6320,-0.493093,0.039271,-0.485255,-0.475820
4,51,6388,-1.423154,-0.385962,-1.329622,-0.896342
...,...,...,...,...,...,...
1889316,74163,74164,1.403953,0.782408,0.458573,1.096325
1889317,74163,74200,0.339694,-0.496990,0.093199,1.169474
1889318,74164,74194,0.878531,0.895928,1.543876,-0.604091
1889319,74164,74200,-0.682904,-0.405051,-0.144396,-0.433730


In [255]:
scaler.mean_

array([1639.53508673,   81.51025658,   94.39842333,  123.00922938])

In [220]:
df = pd.DataFrame([[1000, 100, 80, 15]],columns=target_columns)
df

,energia,grasa,proteina,carbohidratos
0,1000,100,80,15


In [221]:
df[target_columns] = scaler.transform(df[target_columns])
df

,energia,grasa,proteina,carbohidratos
0,-0.93911,-0.531425,-0.533359,-1.071159


In [256]:
result = scaler.transform([[1000, 100, 45, 15]])
result


array([[-1.17155005,  0.40199432, -0.89100108, -1.30756094]])

In [219]:
target_values = np.array([1000, 100, 80, 15])
target_columns = ["energia","grasa",'proteina',"carbohidratos"]

In [257]:
def getClosest(target_values):
    objetivos = scaler.transform([target_values])
    eucl_dist = euclidean_distances(objetivos, data[target_columns].values).reshape(-1) #NP Con las distancias al objetivo
    ind_min_ed = (np.where(eucl_dist == np.amin(eucl_dist)))[0] #indice de la combinacion con menor distancia
    ind_comida = int(data.iloc[int(ind_min_ed)]['Recipe1']) #indices de las recetas 1 y 2 de la combinación
    ind_cena = int(data.iloc[int(ind_min_ed)]['Recipe2'])
    return ind_min_ed, ind_comida, ind_cena

def getClosest2(target_values):
    objetivos = scaler.transform([target_values])
    data['distance'] = euclidean_distances(objetivos, data_values.values).reshape(-1)
    ids = data.nsmallest(1, 'distance')[['Recipe1','Recipe2']]
    return ind_min_ed, ids

def getClosest3(target_values):
    data['distance'] = data.apply(lambda row: euclidean_distances(scaler.transform([target_values]), data_values.values).reshape(-1))
    ids = data.nsmallest(1, 'distance')[['Recipe1','Recipe2']]
    return ind_min_ed, ids

In [261]:
%%timeit
getClosest(target_values) # numpy

77.1 ms ± 11.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [262]:
%%timeit
getClosest2(target_values) # creando una columna pero usando numpy

109 ms ± 682 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [263]:
%%timeit
getClosest3(target_values) # con apply

456 ms ± 9.37 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
